The raw electricity usage data was provided in an unsorted and poorly structured format. As part of the data cleaning process, the dataset was first sorted chronologically by date and hour, missing values were inspected, and data consistency was validated. The cleaned data was then aggregated into two separate datasets: a monthly-level dataset used for analyzing monthly-based pricing schemes, and an hourly-level dataset used for evaluating time-dependent electricity tariffs. These cleaned datasets were saved as monthly_dc.xlsx and hourly_dc.xlsx and served as the foundation for subsequent analysis.


1. Data Cleaning
2. Data Preprocessing
3. EDA
4. Cost Calculation
5. Optimization (Contract Selection)
6. Conclusion

In [2]:
import pandas as pd

monthly_usage = pd.read_excel('/content/monthly_dc.xlsx')
hourly_usage = pd.read_excel('/content/hourly_dc.xlsx')

monthly_usage.head(), hourly_usage.head()

(  monthly_label  monthly_price monthly_unit
 0       January           0.20        $/kWh
 1      February           0.19        $/kWh
 2         March           0.17        $/kWh
 3         April           0.18        $/kWh
 4           May           0.22        $/kWh,
      hourly_label  hourly_price hourly_unit
 0  12AM until 1AM          0.10       $/kWh
 1   1AM until 2AM          0.10       $/kWh
 2   2AM until 3AM          0.10       $/kWh
 3   3AM until 4AM          0.10       $/kWh
 4   4AM until 5AM          0.15       $/kWh)

In [3]:
monthly_usage

,monthly_label,monthly_price,monthly_unit
0,January,0.20,$/kWh
1,February,0.19,$/kWh
2,March,0.17,$/kWh
3,April,0.18,$/kWh
4,May,0.22,$/kWh
5,June,0.27,$/kWh
6,July,0.24,$/kWh
7,August,0.19,$/kWh
8,September,0.18,$/kWh
9,October,0.15,$/kWh


In [4]:
hourly_usage

,hourly_label,hourly_price,hourly_unit
0,12AM until 1AM,0.10,$/kWh
1,1AM until 2AM,0.10,$/kWh
2,2AM until 3AM,0.10,$/kWh
3,3AM until 4AM,0.10,$/kWh
4,4AM until 5AM,0.15,$/kWh
5,5AM until 6AM,0.20,$/kWh
6,6AM until 7AM,0.24,$/kWh
7,7AM until 8AM,0.24,$/kWh
8,8AM until 9AM,0.26,$/kWh
9,9AM until 10AM,0.20,$/kWh


In [5]:
usage = pd.read_csv('/content/data_cleaning .xlsx - Usage.csv')
usage

,3 PM Mon 24th-Mar-2014___0.384 kwh
0,5AM 15-Aug-2014___1.201 kwh
1,__8PM Thu 20-Mar-2014____1.523 kwh
2,6PM 23rd-Apr-2014___0.424 kwh
3,_1AM Friday 19th-Dec-2014___0.209 kwh
4,_5AM Tue 19th-Aug-2014___1.228 kwh
...,...
8754,_1AM Fri 07th-Nov-2014_0.084 kwh
8755,_6AM 20-May-2014__1.027 kwh
8756,__2 AM Tuesday 8th-Apr-2014___0.052 kwh
8757,9 PM 27th-Jan-2014_1.428 kwh


In [6]:
usage.columns

Index([' 3 PM  Mon 24th-Mar-2014___0.384 kwh  '], dtype='object')

In [7]:
usage.columns = ['raw']
usage

,raw
0,5AM 15-Aug-2014___1.201 kwh
1,__8PM Thu 20-Mar-2014____1.523 kwh
2,6PM 23rd-Apr-2014___0.424 kwh
3,_1AM Friday 19th-Dec-2014___0.209 kwh
4,_5AM Tue 19th-Aug-2014___1.228 kwh
...,...
8754,_1AM Fri 07th-Nov-2014_0.084 kwh
8755,_6AM 20-May-2014__1.027 kwh
8756,__2 AM Tuesday 8th-Apr-2014___0.052 kwh
8757,9 PM 27th-Jan-2014_1.428 kwh


In [8]:
usage = usage.copy()
usage.head()


,raw
0,5AM 15-Aug-2014___1.201 kwh
1,__8PM Thu 20-Mar-2014____1.523 kwh
2,6PM 23rd-Apr-2014___0.424 kwh
3,_1AM Friday 19th-Dec-2014___0.209 kwh
4,_5AM Tue 19th-Aug-2014___1.228 kwh


In [9]:
usage["usage"] = (
    usage["raw"]
    .str.extract(r"([0-9]+\.[0-9]+)\s*kwh", expand=False)
    .astype(float)
)


In [10]:
usage

,raw,usage
0,5AM 15-Aug-2014___1.201 kwh,1.201
1,__8PM Thu 20-Mar-2014____1.523 kwh,1.523
2,6PM 23rd-Apr-2014___0.424 kwh,0.424
3,_1AM Friday 19th-Dec-2014___0.209 kwh,0.209
4,_5AM Tue 19th-Aug-2014___1.228 kwh,1.228
...,...,...
8754,_1AM Fri 07th-Nov-2014_0.084 kwh,0.084
8755,_6AM 20-May-2014__1.027 kwh,1.027
8756,__2 AM Tuesday 8th-Apr-2014___0.052 kwh,0.052
8757,9 PM 27th-Jan-2014_1.428 kwh,1.428


In [11]:
usage["unit"] = (
    usage["raw"]
    .str.extract(r"(kwh)", expand=False)
)
usage


,raw,usage,unit
0,5AM 15-Aug-2014___1.201 kwh,1.201,kwh
1,__8PM Thu 20-Mar-2014____1.523 kwh,1.523,kwh
2,6PM 23rd-Apr-2014___0.424 kwh,0.424,kwh
3,_1AM Friday 19th-Dec-2014___0.209 kwh,0.209,kwh
4,_5AM Tue 19th-Aug-2014___1.228 kwh,1.228,kwh
...,...,...,...
8754,_1AM Fri 07th-Nov-2014_0.084 kwh,0.084,kwh
8755,_6AM 20-May-2014__1.027 kwh,1.027,kwh
8756,__2 AM Tuesday 8th-Apr-2014___0.052 kwh,0.052,kwh
8757,9 PM 27th-Jan-2014_1.428 kwh,1.428,kwh


In [12]:
usage["date"] = (
    usage["raw"]
    .str.extract(
        r"((?:\d{1,2})(?:st|nd|rd|th)?-[A-Za-z]{3}-\d{4})",
        expand=False
    )
)
usage["date"] = (
    usage["date"]
    .str.replace(r"(st|nd|rd|th)", "", regex=True)
)
usage["date"] = pd.to_datetime(
    usage["date"],
    format="%d-%b-%Y"
)


In [13]:
usage

,raw,usage,unit,date
0,5AM 15-Aug-2014___1.201 kwh,1.201,kwh,2014-08-15
1,__8PM Thu 20-Mar-2014____1.523 kwh,1.523,kwh,2014-03-20
2,6PM 23rd-Apr-2014___0.424 kwh,0.424,kwh,2014-04-23
3,_1AM Friday 19th-Dec-2014___0.209 kwh,0.209,kwh,2014-12-19
4,_5AM Tue 19th-Aug-2014___1.228 kwh,1.228,kwh,2014-08-19
...,...,...,...,...
8754,_1AM Fri 07th-Nov-2014_0.084 kwh,0.084,kwh,2014-11-07
8755,_6AM 20-May-2014__1.027 kwh,1.027,kwh,2014-05-20
8756,__2 AM Tuesday 8th-Apr-2014___0.052 kwh,0.052,kwh,2014-04-08
8757,9 PM 27th-Jan-2014_1.428 kwh,1.428,kwh,2014-01-27


In [14]:
usage['time'] = usage['raw'].str.extract(r'(\d{1,2})\s*(?=AM|PM)', expand=False)


In [15]:
usage

,raw,usage,unit,date,time
0,5AM 15-Aug-2014___1.201 kwh,1.201,kwh,2014-08-15,5
1,__8PM Thu 20-Mar-2014____1.523 kwh,1.523,kwh,2014-03-20,8
2,6PM 23rd-Apr-2014___0.424 kwh,0.424,kwh,2014-04-23,6
3,_1AM Friday 19th-Dec-2014___0.209 kwh,0.209,kwh,2014-12-19,1
4,_5AM Tue 19th-Aug-2014___1.228 kwh,1.228,kwh,2014-08-19,5
...,...,...,...,...,...
8754,_1AM Fri 07th-Nov-2014_0.084 kwh,0.084,kwh,2014-11-07,1
8755,_6AM 20-May-2014__1.027 kwh,1.027,kwh,2014-05-20,6
8756,__2 AM Tuesday 8th-Apr-2014___0.052 kwh,0.052,kwh,2014-04-08,2
8757,9 PM 27th-Jan-2014_1.428 kwh,1.428,kwh,2014-01-27,9


In [16]:
usage['am-pm'] = usage['raw'].str.extract(r'(AM|PM)', expand=False)

In [17]:
usage

,raw,usage,unit,date,time,am-pm
0,5AM 15-Aug-2014___1.201 kwh,1.201,kwh,2014-08-15,5,AM
1,__8PM Thu 20-Mar-2014____1.523 kwh,1.523,kwh,2014-03-20,8,PM
2,6PM 23rd-Apr-2014___0.424 kwh,0.424,kwh,2014-04-23,6,PM
3,_1AM Friday 19th-Dec-2014___0.209 kwh,0.209,kwh,2014-12-19,1,AM
4,_5AM Tue 19th-Aug-2014___1.228 kwh,1.228,kwh,2014-08-19,5,AM
...,...,...,...,...,...,...
8754,_1AM Fri 07th-Nov-2014_0.084 kwh,0.084,kwh,2014-11-07,1,AM
8755,_6AM 20-May-2014__1.027 kwh,1.027,kwh,2014-05-20,6,AM
8756,__2 AM Tuesday 8th-Apr-2014___0.052 kwh,0.052,kwh,2014-04-08,2,AM
8757,9 PM 27th-Jan-2014_1.428 kwh,1.428,kwh,2014-01-27,9,PM


In [18]:
usage = usage.drop(columns=['raw'])
usage = usage[['time', 'am-pm', 'date', 'usage', 'unit']]
usage

,time,am-pm,date,usage,unit
0,5,AM,2014-08-15,1.201,kwh
1,8,PM,2014-03-20,1.523,kwh
2,6,PM,2014-04-23,0.424,kwh
3,1,AM,2014-12-19,0.209,kwh
4,5,AM,2014-08-19,1.228,kwh
...,...,...,...,...,...
8754,1,AM,2014-11-07,0.084,kwh
8755,6,AM,2014-05-20,1.027,kwh
8756,2,AM,2014-04-08,0.052,kwh
8757,9,PM,2014-01-27,1.428,kwh


In [19]:

usage.head(20)

,time,am-pm,date,usage,unit
0,5,AM,2014-08-15,1.201,kwh
1,8,PM,2014-03-20,1.523,kwh
2,6,PM,2014-04-23,0.424,kwh
3,1,AM,2014-12-19,0.209,kwh
4,5,AM,2014-08-19,1.228,kwh
5,12,PM,2014-07-07,1.296,kwh
6,7,AM,2014-11-25,0.437,kwh
7,8,AM,2014-08-14,0.523,kwh
8,4,PM,2014-01-25,2.052,kwh
9,4,PM,2014-02-11,0.509,kwh


question 1
answer:B

In [20]:
avg_usage = usage['usage'].mean()
avg_usage

np.float64(0.7817935837424365)

question 2
answer : D

In [21]:
usage_1 = usage[usage['date'].dt.month == 2]
usage_1['usage'].mean()

np.float64(0.8337347242921014)

question 3
answer : A

In [22]:
usage['date'].dtype

dtype('<M8[ns]')

In [23]:
usage["dayname"] = usage["date"].dt.day_name()


In [24]:
usage.head(20)

,time,am-pm,date,usage,unit,dayname
0,5,AM,2014-08-15,1.201,kwh,Friday
1,8,PM,2014-03-20,1.523,kwh,Thursday
2,6,PM,2014-04-23,0.424,kwh,Wednesday
3,1,AM,2014-12-19,0.209,kwh,Friday
4,5,AM,2014-08-19,1.228,kwh,Tuesday
5,12,PM,2014-07-07,1.296,kwh,Monday
6,7,AM,2014-11-25,0.437,kwh,Tuesday
7,8,AM,2014-08-14,0.523,kwh,Thursday
8,4,PM,2014-01-25,2.052,kwh,Saturday
9,4,PM,2014-02-11,0.509,kwh,Tuesday


In [25]:
avg_dayname = usage.groupby('dayname')['usage'].mean().sort_values(ascending=False)
avg_dayname

,usage
dayname,
Sunday,0.898601
Saturday,0.890300
Friday,0.753666
Thursday,0.746375
Wednesday,0.733699
Monday,0.726168
Tuesday,0.724843


question 4
answer :

In [26]:
usage = usage.sort_values(by=['usage', 'time','date'], ascending=False )
usage.head(20)

,time,am-pm,date,usage,unit,dayname
2964,8,PM,2014-07-27,5.141,kwh,Sunday
8257,8,PM,2014-07-20,5.141,kwh,Sunday
1738,7,PM,2014-07-26,5.141,kwh,Saturday
1980,7,PM,2014-07-30,5.054,kwh,Wednesday
3618,7,PM,2014-07-20,5.054,kwh,Sunday
3319,7,PM,2014-07-07,5.054,kwh,Monday
2558,6,PM,2014-07-27,5.054,kwh,Sunday
2835,6,PM,2014-07-19,5.054,kwh,Saturday
2230,7,PM,2014-07-28,5.011,kwh,Monday
2860,7,PM,2014-07-25,4.925,kwh,Friday


In [29]:
usage_sorted = usage.sort_values(["date", "time"])
usage_sorted["rolling_4h"] = usage_sorted["usage"].rolling(4).sum()
usage_sorted["rolling_4h"].max()


16.007

question 5
answer : B

In [30]:
usage["month"] = usage["date"].dt.month


In [31]:
usage

,time,am-pm,date,usage,unit,dayname,month
2964,8,PM,2014-07-27,5.141,kwh,Sunday,7
8257,8,PM,2014-07-20,5.141,kwh,Sunday,7
1738,7,PM,2014-07-26,5.141,kwh,Saturday,7
1980,7,PM,2014-07-30,5.054,kwh,Wednesday,7
3618,7,PM,2014-07-20,5.054,kwh,Sunday,7
...,...,...,...,...,...,...,...
2910,2,AM,2014-04-14,0.049,kwh,Monday,4
8526,2,AM,2014-04-10,0.049,kwh,Thursday,4
6343,2,AM,2014-04-07,0.049,kwh,Monday,4
698,4,AM,2014-04-18,0.048,kwh,Friday,4


In [32]:
monthly_prices = {
    1: 0.20,
    2: 0.19,
    3: 0.17,
    4: 0.18,
    5: 0.22,
    6: 0.27,
    7: 0.24,
    8: 0.19,
    9: 0.18,
    10: 0.15,
    11: 0.14,
    12: 0.19
}


In [33]:
usage["monthly_price"] = usage["month"].map(monthly_prices)
usage

,time,am-pm,date,usage,unit,dayname,month,monthly_price
2964,8,PM,2014-07-27,5.141,kwh,Sunday,7,0.24
8257,8,PM,2014-07-20,5.141,kwh,Sunday,7,0.24
1738,7,PM,2014-07-26,5.141,kwh,Saturday,7,0.24
1980,7,PM,2014-07-30,5.054,kwh,Wednesday,7,0.24
3618,7,PM,2014-07-20,5.054,kwh,Sunday,7,0.24
...,...,...,...,...,...,...,...,...
2910,2,AM,2014-04-14,0.049,kwh,Monday,4,0.18
8526,2,AM,2014-04-10,0.049,kwh,Thursday,4,0.18
6343,2,AM,2014-04-07,0.049,kwh,Monday,4,0.18
698,4,AM,2014-04-18,0.048,kwh,Friday,4,0.18


In [34]:
annual_cost_monthly_flex = (usage["usage"] * usage["monthly_price"]).sum()
annual_cost_monthly_flex


np.float64(1421.14049)

Question 6
answer :

In [ ]:
Based on your historic electricity
usage, which of the three contracts
would pro- duce the lowest annual
cost?
a.The No Flex plan
b. The Monthly Flex plan
c. The Hourly Flex plan
d. Impossible to Determine